# ilanları bulma scripti

In [1]:
from contextlib import nullcontext
import time
import random
from bs4.builder import HTML
import pandas as pd
from bs4 import BeautifulSoup # daha ilkel bir html parse kutuphanesi
from selenium import webdriver

Gerekli olan kütüphaneleri yükledik.

In [2]:
driver_path = "C:/webdriver/chromedriver.exe" # webdriver.exe konumunu göstermeliyiz.
driver = webdriver.Chrome(driver_path)
#

Selenium'un çalışması için chrome'a ait webdriver.exe dosyasını tanıttık artık otomatik test yazılımımız çalışıyor.

In [3]:
#%%
driver.get("https://www.kariyer.net/is-ilanlari/?kw=istatistik&sf=1d&is=1") #ana sayfa


Burada kariyer.net 'istatistik' anahtar kelimelerini içeren sayfaya erişmiş bulunmaktayız.ilanlar yeniden eskiye doğru sıralanmaz ise aynı ilan bir kaç defa görülebilmektedir. Bu sebeble bu url'nin düzgün girilmesi daha fazla ilana ulaşamamız için önem teşkil etmektedir.

In [12]:
#%%
nextpageXpath='//*[@id="__layout"]/div/div[2]/div/div[1]/div[2]/div[2]/div[2]/div[3]/div[26]/div/ul/li[14]/button'
from selenium.webdriver.common.action_chains import ActionChains
def nextpage():
    ActionChains(driver).move_to_element(driver.find_element_by_xpath(nextpageXpath)).perform()
    driver.find_element_by_xpath(nextpageXpath).click()
#%%

nextpage bir sonraki sayfaya geçmemizi sağlayan bir fonksiyondur.

In [13]:
a=None
pageLinks=[]
driver.get("https://www.kariyer.net/is-ilanlari/?kw=istatistik&sf=1d&is=1")
while(a==None):
    try:
        pageLinks.append(driver.current_url)
        nextpage()
        time.sleep(1)
    except:
        a="stop"
        print(a)
        break
        

stop


Burada bir ilanların sayfa linklerine erişip bunları bir değişkene atadık.

In [7]:
#__layout > div > div.content-wrapper > div > div.k-skeleton-joblist.mt-lg-0 > div.clean-container-padding.container > div:nth-child(2) > div.col > div.list-items-wrapper > div:nth-child(1)
driver.find_elements_by_css_selector('div.list-items')[2].find_element_by_tag_name('a').get_attribute('href')

'https://www.kariyer.net/is-ilani/lc-waikiki-magazacilik-hizmetleri-tic-a-s-master-data-veri-analizi-uzmani-2663975'

Görüldüğü gibi burada her bir ilana ait olan linki alabildiğimizi tek bir ilan için görmekteyiz. Şimdi bu durumu her bir sayfadaki tüm ilan linklerini bulmak için genelleştirelim.

In [8]:
urls=[]
for page in pageLinks:
    driver.get(page)
    time.sleep(3)
    ilanElements=driver.find_elements_by_css_selector('div.list-items')
    for ilan in ilanElements:
        urls.append(ilan.find_element_by_tag_name('a').get_attribute('href'))  

In [13]:
urls

['https://www.kariyer.net/is-ilani/vakif-pazarlama-sanayi-ve-ticaret-a-s-is-yeri-hekimi-kayseri-2761428',
 'https://www.kariyer.net/is-ilani/biofarma-ilac-butce-ve-raporlama-uzmani-kidemli-uzmani-2765378',
 'https://www.kariyer.net/is-ilani/destek-faktoring-a-s-banka-yetkilisi-yonetmeni-2765310',
 'https://www.kariyer.net/is-ilani/ck-bogazici-elektrik-perakende-satis-a-s-satis-planlama-uzman-yardimcisi-uzmani-2765263',
 'https://www.kariyer.net/is-ilani/adresgezgini-a-s-portfoy-yoneticisi-2765211',
 'https://www.kariyer.net/is-ilani/istanbul-favori-gida-san-tic-a-s-satis-analiz-ve-raporlama-sorumlusu-2388400',
 'https://www.kariyer.net/is-ilani/cetas-otomotiv-crm-ve-musteri-analitigi-muduru-2630588',
 'https://www.kariyer.net/is-ilani/asis-otomasyon-ve-akaryakit-sistemleri-a-s-veri-analiz-uzmani-1086432',
 'https://www.kariyer.net/is-ilani/koton-magazacilik-tekstil-san-ve-tic-a-s-analiz-raporlama-uzmani-sorumlusu-2765042',
 'https://www.kariyer.net/is-ilani/astranova-tarim-tic-ve-san-a

Burada analizde potansiyel olarak kullanacağımız tüm ilan linklerine erişmiş bulunmaktayız.

In [14]:
import pickle

In [15]:
with open('urls.pkl', 'wb') as f:
    pickle.dump(urls, f)

Burada ise url variable'ını farklı bir scripte kullanabilmek için pickle ile saklıyoruz. Farklı bir scriptte bu ilanlara ait olan tüm ilan içeriklerini çekeceğiz.